In [ ]:
import tensorflow as tf

## 1. Build the model

In [2]:
class Model():
    def __init__(self):
        self.W = tf.Variable(5.0)
        self.b = tf.Variable(0.0)
    
    def __call__(self, x):
        return sel.W * x + self.b

## 2. Prepare training Data

In [4]:
True_w = 3.0
True_b = 2.0
NUM_EX = 1000
xs = tf.random.normal(shape=[NUM_EX])
yx = True_w * xs + True_b

## 3. Mean Squared Error Loss

In [6]:
def mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

## 4. Calculate Partial Derivative of Loss

In [ ]:
def train(model, xs, ys, lr=0.01):
    with tf.GradientTape() as tape:
        y_pred = model(inputs)
        loss = mse(ys, y_pred)
    w_grad = tape.gradient(loss, )